# Partition Crossover for Steiner Trees (PXST) - How does it work?

This notebook aims to explain the Partition Crossover Algorithm's main steps for the Steiner Tree Problem in Graphs.

The first version is available in the package:

```python
from ga4stpg.tree.pxcrossover import PXTree
```

## Imports

The packages used in the implementation:

In [1]:
from collections import deque
from operator import attrgetter
from random import choice

from ga4stpg.graph import UGraph
from ga4stpg.graph.disjointsets import DisjointSets

## Component Class

This class represents a component (or partition) formed by a non-common edges' set.

The edges are stored in a dictionary data structure. Then, we maintain the correspondent parent node information for each edge, although this information is useless for the rest of the algorithm.

The add method does not guarantee that component will be all connected.

In [2]:
class Component:

    def __init__(self, first, second, initialcost=0):
        self.edges = {first : None, second : first}
        self.portal = set()
        self.cost = initialcost

    def add(self, first, second):
        self.edges[second] = first

    def __contains__(self, value):
        return value in self.edges

    def __str__(self):
        return self.__repr__()

    def __repr__(self):
        return f"Component <{self.portal}>"

## Compose function

A compose function returns three subgraphs that will be used forward: the union subgraph; the subgraph formed only by common edges; and the star subgraph, the subgraph formed only by non-common edges.

In [3]:
def compose(red : UGraph, blue : UGraph):
    '''
    Parameters:
    ----------
        red, blue : Graph

    Return:
    -------
        g_union, g_common, g_star : Graph
    '''

    g_union  = UGraph()
    g_common = UGraph()
    g_star   = UGraph()

    for v, u in red.gen_undirect_edges():
        g_union.add_edge(v, u)

        if not blue.has_edge(v,u):
            g_star.add_edge(v,u)

    for v, u in blue.gen_undirect_edges():
        g_union.add_edge(v, u)

        if red.has_edge(v,u):
            g_common.add_edge(v, u)
        else:
            g_star.add_edge(v,u)


    return g_union, g_common, g_star

## Indentifying the components


In this version, the components are identified from the union graph. 
The common graph is the base for the offspring that the operation will return it. 

The star graph is not used at all. 
Some explications for this fact can be:
1. The transversal search on the union graph builds the disjoint sets for the common edges as its finds the components;
2. The implementation of the transversal search does not handle disconnected components.

The method "connected" computes the components for recombination. Each component will be tested and evaluated later.

This method takes the following parameters: the union graph, the red solution (Steiner tree); the blue solution; and a start node. The start vertice is always a terminal node.

```python
def connected(self, g_union : UGraph, red : UGraph, blue : UGraph, start : 'node'):
    #...
```

Furthermore, it returns a list of components for the red solution and another list for the blue solution. This method also returns a disjoint set data structure representing the components formed by joint edges.


 

In [4]:
# def connected(self, g_union : UGraph, red : UGraph, blue : UGraph, start : 'node'):
def connected( g_union : UGraph, red : UGraph, blue : UGraph, start : 'node', f-f_weight):

        linkedlist = deque([start])
        visited = set()
        parents = DisjointSets()
        # f_weight = f_weight

        for v in g_union.vertices:
            parents.make_set(v)

        def chase(main, node, component):

            if (node in red) and (node in blue):
                if node not in visited and node not in linkedlist:
                    linkedlist.appendleft(node)

                component.portal.add(node)
                return

            for w in main.adjacent_to(node):
                if w in component.edges:
                    continue
                component.add(node, w)
                component.cost += f_weight(node, w)
                chase(main, w, component)
            # end for loop
            visited.add(node)

        components_red  = list()
        components_blue = list()

        while linkedlist:
            u = linkedlist.pop()
            visited.add(u)
            for v in g_union.adjacent_to(u):
                if v in visited:
                    continue
                if red.has_edge(u,v) and blue.has_edge(u,v):
                    linkedlist.append(v)
                    parents.union(u, v)

                elif red.has_edge(u,v):
                    component = Component(u, v, initialcost=f_weight(u,v))
                    chase(red, v, component)
                    chase(red, u, component)
                    components_red.append(component)

                elif blue.has_edge(u,v):
                    component = Component(u, v, initialcost=f_weight(u,v))
                    chase(blue, v, component)
                    chase(blue, u, component)
                    components_blue.append(component)
            # end for loop
        # end while loop
        return components_red, components_blue, parents

For this example, let us consider the following STPG instance and some candidate solutions. 

In [5]:
from os import path
from pprint import pprint
from ga4stpg.graph.reader import ReaderORLibrary

instance_problem = "steinb5.txt"
folder_datasets = path.join('..', '..', 'ppgi-stpg-gpx', 'datasets', 'ORLibrary')
filename = path.join(folder_datasets, instance_problem)

STPG = ReaderORLibrary().parser(filename)

def f_weight(x, y):
    return STPG.graph.weight(x,y)

In [6]:
print("STPG information", '\n', 10*'- ','\n')
print('Instance: ', STPG.name)
print("Nro. Node:", STPG.nro_nodes)
print("Nro. Edges:", STPG.nro_edges)
print("Nro. Terminals:", STPG.nro_terminals)
print("Terminals: \n", STPG.terminals)

STPG information 
 - - - - - - - - - -  

Instance:  B5
Nro. Node: 50
Nro. Edges: 100
Nro. Terminals: 13
Terminals: 
 {3, 35, 5, 37, 7, 39, 13, 15, 16, 20, 23, 24, 31}


In [7]:
from ga4stpg.tree.evaluation import EvaluateTreeGraph
from ga4stpg.tree.generate import GenerateBasedRandomWalk
from ga4stpg.tree.mutate import Prunning

generator = GenerateBasedRandomWalk(STPG)
evaluator = EvaluateTreeGraph(STPG)
prunning  = Prunning(STPG)

red  = generator()
blue = generator()

print(evaluator(red))
print(evaluator(blue))


(282, 1)
(252, 1)


In [8]:
red = prunning(red)
blue = prunning(blue)

print(evaluator(red))
print(evaluator(blue))

(167, 1)
(163, 1)


In [9]:
g_union, g_common, g_star = compose(red, blue)

In [10]:
from ga4stpg.graph.util import how_many_components

print(how_many_components(g_common))
print(how_many_components(g_union))
print(how_many_components(g_star))

7
1
1


In [11]:
start = choice(tuple(STPG.terminals))
print("starting node: ", start)

first, second, previous = connected(g_union, red, blue, start, f_weight)
print(type(first), len(first))
print(type(second), len(second))
print(type(previous), len(previous))

starting node:  16
<class 'list'> 15
<class 'list'> 14
<class 'ga4stpg.graph.disjointsets.DisjointSets'> 40


In [12]:
# for component in first:
#     print(component.portal," Cost: ",component.cost)

In [13]:
# for component in second:
#     print(component.portal," Cost: ",component.cost)

In [14]:
print("How many components are there?")

print(len(first))
print(len(second))

How many components are there?
15
14


In [15]:
from collections import defaultdict

In [16]:
candidates = defaultdict(list)
recombinantes = list()
non_first = list()
non_second = list()

for component in first:
    ss = frozenset(previous.find(v) for v in component.portal)
    candidates[ss].append(component)

for component in second:
    ss = frozenset(previous.find(v) for v in component.portal)
    if ss in candidates:
        candidates[ss].append(component)
    else:
        non_second.append(component)

for values in candidates.values():
    if len(values) == 2 :
        recombinantes.append(values)
    else:
        non_first.append(values)

In [17]:
pprint(candidates)

defaultdict(<class 'list'>,
            {frozenset({2, 20}): [Component <{2, 20}>],
             frozenset({1, 3}): [Component <{1, 21}>],
             frozenset({3, 39}): [Component <{3, 39}>],
             frozenset({4, 37}): [Component <{4, 37}>],
             frozenset({35, 4}): [Component <{49, 4}>],
             frozenset({35, 20}): [Component <{49, 20}>],
             frozenset({1, 7}): [Component <{1, 7}>],
             frozenset({1, 10}): [Component <{40, 1}>, Component <{1, 18}>],
             frozenset({24, 15}): [Component <{24, 25}>],
             frozenset({37, 45}): [Component <{36, 37}>],
             frozenset({35, 27}): [Component <{17, 35}>],
             frozenset({24, 45}): [Component <{24, 45}>],
             frozenset({10, 5}): [Component <{18, 5}>, Component <{18, 5}>],
             frozenset({27, 5}): [Component <{48, 5}>]})


In [18]:
recombinantes

[[Component <{40, 1}>, Component <{1, 18}>],
 [Component <{18, 5}>, Component <{18, 5}>]]

In [19]:
print("Recombinants: ", len(recombinantes))
print("Non recombinants for first: ", len(non_first))
print("Non recombinants for second:", len(non_second))

Recombinants:  2
Non recombinants for first:  12
Non recombinants for second: 13


As we can observe, just a few components are feasible for recombination following the most straightforward criterion, which is components whose portal vertices connect to the same common edges component.

In [20]:
# Util

def list_of_edges(component):
    edges = set()
    for u, v in component.edges.items():
        if v is not None: 
            edges.add((u, v))
    return edges

In [21]:
red_components = list()

for component in first:
    edges = list_of_edges(component)
    # existe alguma aresta dos componentes de 'first' que não pertença a 'red'
    result = all(red.has_edge(v,u) for v, u in edges)
    # existe alguma aresta de algum componente em 'first' que tbm pertença ao grafo 'blue'
    r_blue = any(blue.has_edge(v,u) for v, u in edges)
    print(component, (result and not r_blue))

Component <{2, 20}> True
Component <{1, 21}> True
Component <{3, 39}> True
Component <{4, 37}> True
Component <{49, 4}> True
Component <{49, 20}> True
Component <{1, 7}> True
Component <{40, 1}> True
Component <{24, 25}> True
Component <{36, 37}> True
Component <{17, 35}> True
Component <{24, 45}> True
Component <{18, 5}> True
Component <{18, 5}> True
Component <{48, 5}> True


In [22]:
for component in second:
    edges = list_of_edges(component)
    result = all(blue.has_edge(v,u) for v, u in edges)
    r_red = any(red.has_edge(v,u) for v, u in edges)
    print(component, (result and not r_red))

Component <{16, 21}> True
Component <{2, 4}> True
Component <{3, 20}> True
Component <{25, 23}> True
Component <{24, 4}> True
Component <{36, 4}> True
Component <{20, 39}> True
Component <{1, 35}> True
Component <{1, 18}> True
Component <{48, 25}> True
Component <{40, 37}> True
Component <{49, 5}> True
Component <{49, 45}> True
Component <{17, 7}> True


In [23]:
# existe alguma aresta pertencente aos componentes  que pertença ao grafo 'g_common'
# isto é, alguma aresta que pertença a ambas as soluções?

for component in first:
    edges = list_of_edges(component)
    result = any(g_common.has_edge(v,u) for v, u in edges)
    print(component," Cost: ",component.cost, result)

print("\n\n")

for component in second:
    edges = list_of_edges(component)
    result = any(g_common.has_edge(v,u) for v, u in edges)
    print(component," Cost: ",component.cost, result)

Component <{2, 20}>  Cost:  4 False
Component <{1, 21}>  Cost:  5 False
Component <{3, 39}>  Cost:  4 False
Component <{4, 37}>  Cost:  9 False
Component <{49, 4}>  Cost:  10 False
Component <{49, 20}>  Cost:  2 False
Component <{1, 7}>  Cost:  2 False
Component <{40, 1}>  Cost:  8 False
Component <{24, 25}>  Cost:  16 False
Component <{36, 37}>  Cost:  10 False
Component <{17, 35}>  Cost:  16 False
Component <{24, 45}>  Cost:  8 False
Component <{18, 5}>  Cost:  3 False
Component <{18, 5}>  Cost:  3 False
Component <{48, 5}>  Cost:  3 False



Component <{16, 21}>  Cost:  16 False
Component <{2, 4}>  Cost:  5 False
Component <{3, 20}>  Cost:  1 False
Component <{25, 23}>  Cost:  3 False
Component <{24, 4}>  Cost:  1 False
Component <{36, 4}>  Cost:  20 False
Component <{20, 39}>  Cost:  1 False
Component <{1, 35}>  Cost:  10 False
Component <{1, 18}>  Cost:  3 False
Component <{48, 25}>  Cost:  9 False
Component <{40, 37}>  Cost:  4 False
Component <{49, 5}>  Cost:  4 False
Component 

In [26]:
# Os vértices portais são todos vértices comuns às duas soluções?

assert all(red.has_node(p) and blue.has_node(p) 
        for p in component.portal 
            for component in first)

assert all(red.has_node(p) and blue.has_node(p) 
        for p in component.portal 
            for component in second)

In [30]:
# Existem vértices comuns que não são portais no interior das componentes?

vertices_first = set()
for component in first:
    edges = list_of_edges(component)
    for v, u in edges:
        if v not in component.portal:
            vertices_first.add(v)
        if u not in component.portal:
            vertices_first.add(u)

any(red.has_node(v) and blue.has_node(v) for v in vertices_first)

False

In [31]:
vertices_second = set()

for component in second:
    edges = list_of_edges(component)
    for v, u in edges:
        if v not in component.portal:
            vertices_second.add(v)
        if u not in component.portal:
            vertices_second.add(u)

any(red.has_node(v) and blue.has_node(v) for v in vertices_second)

False